In [1]:
#directory structure of this project

# /root/nb/ -> notebooks
# /root/images/raw/ -> images
# /root/labels/ -> classification labels


In [2]:
!pip3 install lxml
!pip install pandas tqdm
import pandas as pd
import numpy as np
import os
import urllib.request as req
from urllib.request import urlopen
import re
from tqdm import tnrange, tqdm_notebook
from time import sleep
import matplotlib.pyplot as plt

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
root_dir = os.getcwd()

In [6]:
!mkdir root_dir + 'pix'

mkdir: cannot create directory ‘root_dir’: File exists
mkdir: cannot create directory ‘+’: File exists
mkdir: cannot create directory ‘pix’: File exists


In [10]:


tables = pd.read_html('http://projects.biodiversity.be/openuprbins/search?page=3')
print(tables[0])
print(tables[1])
print(tables[2])

  # collect the total information split in 3 tables
entry1  = [tables[0], tables[1], tables[2]] 
  # combine the 3 tables into one table
entry1 = pd.concat(entry1)
  # for convenicnce, set the first column to row names (Family,Subspecies,...)
entry1 = entry1.set_index(entry1.columns[0])

entry1

             0            1
0      Family:  Buprestidae
1   Subfamily:            /
2       Genus:    Buprestis
3     Species:      rufipes
4  Subspecies:            /
           0              1
0      Date:              /
1   Country:  United States
2  Province:          Texas
3   Station:              /
           0                                            1
0  Filename:  Buprestidae/Buprestis rufipes 40339cz42.jpg
1      View:                                       Dorsal


,1
0,
Family:,Buprestidae
Subfamily:,/
Genus:,Buprestis
Species:,rufipes
Subspecies:,/
Date:,/
Country:,United States
Province:,Texas
Station:,/


In [ ]:
# 1) PREP DATA
# the data for each entry is always split into 3 tables... we need to combine the 3 tables 
# to get the total information for a given entry
# ASSUMPTION : the relevant tables for each entry is always [0,1,2], [3,4,5], ... [15,16,17]
# ASSUMPTION : Always 6 entries per page

# 2) EXTRACT LABELS
# We assume that the Family, Genus, Species, and View are relevant.
# Only if the View is DORSAL will we collect the labels

# 3) COLLECT IMAGE
# If there is a dorsal view, we will download the image.
# We must find the url in the html code.
# ASSUMPTION : the order of our regex for the files matches order of data 
#                 present on the webpage

# 4) CREATE CSV
# The csv will contain everything needed to do the classification. This is
# namely the images and the labels in a convenient package.

In [17]:
def collectEntry(tables,index):

# tables = pd.read_html('http://projects.biodiversity.be/openuprbins/search?page=1')

    # collect the total information split in 3 tables
    entry = [tables[index],tables[index+1],tables[index+2]]
    # combine the 3 tables into one table
    entry = pd.concat(entry)
    # for convenience, set the first column to row names (Family,Subspecies,...)
    entry = entry.set_index(entry.columns[0])
    # Grabel relevants labels + view
    entry = entry.loc[['Family:',"Subfamily:", "Country:" , "Province:","Genus:","Species:","View:"]]
    entry = entry.T
    entry['Filename:'] = np.nan
    return entry

In [14]:
index=0
entry = [tables[index],tables[index+1],tables[index+2]]
# combine the 3 tables into one table
entry = pd.concat(entry)
# for convenience, set the first column to row names (Family,Subspecies,...)
entry = entry.set_index(entry.columns[0])
# Grabel relevants labels + view
entry = entry.loc[['Family:',"Subfamily:", "Country:" , "Province:","Genus:","Species:","View:"]]
entry = entry.T
entry['Filename:'] = np.nan

In [15]:
entry

,Family:,Subfamily:,Country:,Province:,Genus:,Species:,View:,Filename:
1,Buprestidae,/,United States,Texas,Buprestis,rufipes,Dorsal,NaN


In [18]:
entry1 = collectEntry(tables,0)
entry2 = collectEntry(tables,3)
entry3 = collectEntry(tables,6)

allentry = pd.concat([entry1,entry2,entry3])
allentry

,Family:,Subfamily:,Country:,Province:,Genus:,Species:,View:,Filename:
1,Buprestidae,/,United States,Texas,Buprestis,rufipes,Dorsal,NaN
1,Buprestidae,/,Australia,Queensland,Calodema,kirbyi,Dorsal,NaN
1,Buprestidae,/,Australia,Queensland,Calodema,plebeja,Dorsal,NaN


In [32]:
def getBeetleDataset(URL):
#this is a web scraper customized for this use case.
#each page has 6 images per with taxonomic information and view angle
#if it is dorsal view, then the image and taxonomy will be collected
    tables = pd.read_html(URL)
    relevant_tables = pd.DataFrame()
    file = urlopen(URL)
    html = file.read()
    for i in range(0,17,3):
        entry_oi = collectEntry(tables,i)
        os.chdir(root_dir + '/images/')
        image_order = (int(i/3))
        thumbnails = re.findall(pattern = "http://projects.biodiversity.be/openup/rbins/[0-9]*.jpg", string=str(html))
        image_id = thumbnails[image_order].split('/')[5] # this grabs the jpg name of interest
        entry_oi['Filename:'] = 'rbi_'+image_id
        relevant_tables = pd.concat([relevant_tables,entry_oi])
        image_grab = req.urlretrieve("http://projects.biodiversity.be/openup/rbins/pictures_only/" + image_id, 'rbi_' + image_id)
# print(entry_oi)
    return(relevant_tables)

In [33]:
beetles

,Family:,Subfamily:,Country:,Province:,Genus:,Species:,View:,Filename:
1,Buprestidae,/,Malaysia,/,Chrysochroa,hemixantha,Lateral,rbi_54.jpg


In [37]:

import tqdm
master_csv = pd.DataFrame()
for i in tqdm.notebook.tqdm(range(1,680),ncols=679):
    URL = 'http://projects.biodiversity.be/openuprbins/search?page=' + str(i)
    beetles = getBeetleDataset(URL)
    master_csv = pd.concat([master_csv,beetles])



In [35]:
master_csv

,Family:,Subfamily:,Country:,Province:,Genus:,Species:,View:,Filename:
1,Anthribidae,/,Equatorial Guinea,/,Phaeotragus,gigas,Dorsal,rbi_1.jpg
1,Anthribidae,/,Equatorial Guinea,/,Phaeotragus,gigas,Lateral,rbi_2.jpg
1,Anthribidae,/,Madagascar,/,Tophroderes,phrenatus,Dorsal,rbi_3.jpg
1,Brentidae,/,Indonesia,Sumatra,Eutrachelus,temmincki,Dorsal,rbi_4.jpg
1,Brentidae,/,Indonesia,Sumatra,Eutrachelus,temmincki,Dorsal,rbi_5.jpg
1,Brentidae,/,Indonesia,Sumatra,Eutrachelus,temmincki,Lateral,rbi_6.jpg
1,Buprestidae,/,Uzbekistan,/,Aaata,finchi,Dorsal,rbi_7.jpg
1,Buprestidae,/,Uzbekistan,/,Aaata,finchi,Lateral,rbi_8.jpg
1,Buprestidae,/,Brazil,Santa Catarina,Actenodes,costipennis,Dorsal,rbi_9.jpg
1,Buprestidae,/,Argentina,/,Archardella,americana,Dorsal,rbi_10.jpg


In [39]:
master_csv.to_csv(r'master_rbi.csv', index = None, header=True)